In [1]:
import pandas as pd
import os

import numpy as np

import os
from astropy.table import Table, vstack
import astropy
from astropy.coordinates import SkyCoord
from astropy.time import Time

import getpass

import lsst.afw.display as afwDisplay
import lsst.afw.geom as afwGeom
import lsst.afw.image as afwImage
from lsst.daf.butler import Butler
import lsst.geom as geom

from lsst.ctrl.mpexec import pipeline2dot
from lsst.ctrl.mpexec import SimplePipelineExecutor
from lsst.pipe.base import Pipeline, Instrument

In [2]:
# afwDisplay.setDefaultBackend('matplotlib')
# plt.style.use('tableau-colorblind10')

In [3]:
#load collection with calexp 
my_collection_identifier_step1step2 = 'test_512025_1_t4026p15y2022'
print(my_collection_identifier_step1step2)

user = os.getenv("USER")
print(user)

collections_step1step2 = f"u/{user}_pm/{my_collection_identifier_step1step2}" 
print('Name of new butler collection for my output: ', collections_step1step2)

test_512025_1_t4026p15y2022
taceroc
Name of new butler collection for my output:  u/taceroc_pm/test_512025_1_t4026p15y2022


In [4]:
# load butler
repo = '/global/cfs/cdirs/lsst/production/gen3/DC2/Run2.2i/repo'
butler = Butler(repo, collections=collections_step1step2)

In [5]:
#list calexps
calexp_list = list(butler.registry.queryDataIds(
    ["tract", "patch", "visit", "detector"],
    instrument="LSSTCam-imSim",
    datasets="calexp",
    collections=collections_step1step2,))

In [6]:
len(calexp_list)

4548

In [7]:
bands = np.unique([c['band'] for c in calexp_list], return_counts=True)

In [8]:
bands

(array(['g', 'i', 'r', 'u', 'y', 'z'], dtype='<U1'),
 array([ 488, 1101, 1237,  288,  876,  558]))

In [9]:
visitTableRef = list(butler.registry.queryDatasets('visitTable'))

In [10]:
len(visitTableRef)

1

In [11]:
visitTable = pd.DataFrame()
for v in visitTableRef:
    visitTable_uni = butler.get(v)
    # visitTable_uni = visitTable_uni[visitTable_uni['band'] == 'g']
    visitTable = pd.concat([visitTable,visitTable_uni])

In [12]:
visitTable

,visit,physical_filter,band,ra,dec,decl,skyRotation,azimuth,altitude,zenithDistance,airmass,expTime,expMidpt,expMidptMJD,obsStart,obsStartMJD
visitId,,,,,,,,,,,,,,,,
2337,2337,u_sim_1.4,u,56.364387,-36.496552,-36.496552,157.193467,244.498722,71.766718,18.233282,1.048596,30.0,2022-01-04 02:56:09.676900,59583.122334,2022-01-04 02:55:54.676900,59583.122161
2339,2339,u_sim_1.4,u,59.411523,-34.045152,-34.045152,161.731267,251.216386,74.441734,15.558266,1.034707,30.0,2022-01-04 02:57:27.868900,59583.123239,2022-01-04 02:57:12.868900,59583.123066
5886,5886,y_sim_1.4,y,57.284375,-34.151092,-34.151092,152.767200,248.318036,77.345682,12.654318,1.022493,30.0,2022-01-09 02:14:56.131100,59588.093705,2022-01-09 02:14:41.131100,59588.093532
7997,7997,z_sim_1.4,z,58.753229,-36.580376,-36.580376,89.317449,232.657715,78.790533,11.209467,1.017579,30.0,2022-01-12 01:56:27.532900,59591.080874,2022-01-12 01:56:12.532900,59591.080701
8003,8003,z_sim_1.4,z,57.172645,-34.899490,-34.899490,99.420167,244.587368,77.466879,12.533121,1.022071,30.0,2022-01-12 02:01:02.716900,59591.084059,2022-01-12 02:00:47.716900,59591.083886
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252424,252424,r_sim_1.4,r,56.031139,-34.021489,-34.021489,190.893807,245.602699,79.672668,10.327332,1.014663,30.0,2022-12-16 03:33:45.753800,59929.148446,2022-12-16 03:33:30.753800,59929.148273
254381,254381,g_sim_1.4,g,58.078976,-36.406712,-36.406712,182.297901,212.372001,82.596299,7.403701,1.007525,30.0,2022-12-19 03:04:26.131100,59932.128080,2022-12-19 03:04:11.131100,59932.127907
254407,254407,g_sim_1.4,g,58.411562,-35.460186,-35.460186,207.230852,237.317493,79.520499,10.479501,1.015192,30.0,2022-12-19 03:29:21.542200,59932.145388,2022-12-19 03:29:06.542200,59932.145215


In [13]:
np.unique(visitTable['band'], return_counts=True)

(array(['g', 'i', 'r', 'u', 'y', 'z'], dtype=object),
 array([11, 26, 28,  6, 19, 12]))

In [19]:
df_band = pd.get_dummies(visitTable['band'], dtype=float)

In [20]:
df_band

,g,i,r,u,y,z
visitId,,,,,,
2337,0.0,0.0,0.0,1.0,0.0,0.0
2339,0.0,0.0,0.0,1.0,0.0,0.0
5886,0.0,0.0,0.0,0.0,1.0,0.0
7997,0.0,0.0,0.0,0.0,0.0,1.0
8003,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...
252424,0.0,0.0,1.0,0.0,0.0,0.0
254381,1.0,0.0,0.0,0.0,0.0,0.0
254407,1.0,0.0,0.0,0.0,0.0,0.0


In [21]:
df = pd.concat([visitTable, df_band], axis=1).reindex(visitTable.index)

In [22]:
df

,visit,physical_filter,band,ra,dec,decl,skyRotation,azimuth,altitude,zenithDistance,...,expMidpt,expMidptMJD,obsStart,obsStartMJD,g,i,r,u,y,z
visitId,,,,,,,,,,,,,,,,,,,,,
2337,2337,u_sim_1.4,u,56.364387,-36.496552,-36.496552,157.193467,244.498722,71.766718,18.233282,...,2022-01-04 02:56:09.676900,59583.122334,2022-01-04 02:55:54.676900,59583.122161,0.0,0.0,0.0,1.0,0.0,0.0
2339,2339,u_sim_1.4,u,59.411523,-34.045152,-34.045152,161.731267,251.216386,74.441734,15.558266,...,2022-01-04 02:57:27.868900,59583.123239,2022-01-04 02:57:12.868900,59583.123066,0.0,0.0,0.0,1.0,0.0,0.0
5886,5886,y_sim_1.4,y,57.284375,-34.151092,-34.151092,152.767200,248.318036,77.345682,12.654318,...,2022-01-09 02:14:56.131100,59588.093705,2022-01-09 02:14:41.131100,59588.093532,0.0,0.0,0.0,0.0,1.0,0.0
7997,7997,z_sim_1.4,z,58.753229,-36.580376,-36.580376,89.317449,232.657715,78.790533,11.209467,...,2022-01-12 01:56:27.532900,59591.080874,2022-01-12 01:56:12.532900,59591.080701,0.0,0.0,0.0,0.0,0.0,1.0
8003,8003,z_sim_1.4,z,57.172645,-34.899490,-34.899490,99.420167,244.587368,77.466879,12.533121,...,2022-01-12 02:01:02.716900,59591.084059,2022-01-12 02:00:47.716900,59591.083886,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252424,252424,r_sim_1.4,r,56.031139,-34.021489,-34.021489,190.893807,245.602699,79.672668,10.327332,...,2022-12-16 03:33:45.753800,59929.148446,2022-12-16 03:33:30.753800,59929.148273,0.0,0.0,1.0,0.0,0.0,0.0
254381,254381,g_sim_1.4,g,58.078976,-36.406712,-36.406712,182.297901,212.372001,82.596299,7.403701,...,2022-12-19 03:04:26.131100,59932.128080,2022-12-19 03:04:11.131100,59932.127907,1.0,0.0,0.0,0.0,0.0,0.0
254407,254407,g_sim_1.4,g,58.411562,-35.460186,-35.460186,207.230852,237.317493,79.520499,10.479501,...,2022-12-19 03:29:21.542200,59932.145388,2022-12-19 03:29:06.542200,59932.145215,1.0,0.0,0.0,0.0,0.0,0.0


In [29]:
day_obds = [x['day_obs'] for x in calexp_list]
dates_calexps = pd.DataFrame()
dates_calexps.loc[:, 'year'] = [str(x)[:4] for x in day_obds]
dates_calexps.loc[:, 'month'] = [str(x)[4:6] for x in day_obds]
dates_calexps.loc[:, 'day'] = [str(x)[6:] for x in day_obds]
dates_calexps.loc[:, 'time'] = dates_calexps.loc[:, 'year']+'-'+dates_calexps.loc[:, 'month']+'-'+dates_calexps.loc[:, 'day']
dates_calexps.loc[:, 'time'] = dates_calexps.loc[:, 'time'].map(lambda x: Time(x))
dates_calexps.loc[:, 'mjd'] = dates_calexps.loc[:, 'time'].map(lambda x: x.mjd)
dates_calexps.loc[:, 'mjd'] = dates_calexps.loc[:, 'mjd'].map(lambda x: round(x,0))

dates_calexps_ym = dates_calexps.groupby(["year", "month"]).agg({'mjd':'first', 'time':'count'})
dates_calexps_ym.reset_index(inplace=True)
dates_calexps_ym

,year,month,mjd,time
0,2022,01,59596.0,550
1,2022,02,59629.0,79
2,2022,07,59790.0,70
3,2022,08,59809.0,611
4,2022,09,59839.0,698
5,2022,10,59861.0,1701
6,2022,11,59886.0,395
7,2022,12,59936.0,444


In [18]:
my_range = np.array((visitTable['obsStartMJD'] >= dates_calexps_ym['mjd'][0]) & 
                    (visitTable['obsStartMJD'] < dates_calexps_ym['mjd'][0+1]))
my_visits = visitTable_uni[my_range]

In [19]:
my_visits

,visit,physical_filter,band,ra,dec,decl,skyRotation,azimuth,altitude,zenithDistance,airmass,expTime,expMidpt,expMidptMJD,obsStart,obsStartMJD
visitId,,,,,,,,,,,,,,,,
12453,12453,y_sim_1.4,y,60.271646,-34.760547,-34.760547,232.687741,247.583930,75.494808,14.505192,1.029983,30.0,2022-01-18 01:59:57.657800,59597.083306,2022-01-18 01:59:42.657800,59597.083133
12454,12454,y_sim_1.4,y,57.555769,-33.896871,-33.896871,238.834294,252.348371,73.312992,16.687008,1.040220,30.0,2022-01-18 02:00:36.364900,59597.083754,2022-01-18 02:00:21.364900,59597.083581
12466,12466,y_sim_1.4,y,57.695862,-36.116738,-36.116738,232.526056,246.020643,71.378338,18.621662,1.050812,30.0,2022-01-18 02:08:50.486200,59597.089473,2022-01-18 02:08:35.486200,59597.089300
12482,12482,y_sim_1.4,y,59.119638,-35.607322,-35.607322,229.332616,248.142531,70.472400,19.527600,1.056194,30.0,2022-01-18 02:19:32.697800,59597.096906,2022-01-18 02:19:17.697800,59597.096733


In [20]:
visitTable.shape

(102, 16)

In [25]:
for i in range(0, len(dates_calexps_ym)-1):
    print(i, i+1)
    print(f"{dates_calexps_ym['year'][i]+dates_calexps_ym['month'][i]}-{dates_calexps_ym['month'][i+1]}")
    my_range = np.array((visitTable['obsStartMJD'] >= dates_calexps_ym['mjd'][i]) & 
                    (visitTable['obsStartMJD'] < dates_calexps_ym['mjd'][i+1]))
    my_visits = visitTable_uni.band[my_range]
    print(my_visits)

0 1
202201-02
visitId
12453    y
12454    y
12466    y
12482    y
Name: band, dtype: object
1 2
202202-07
visitId
37656    y
37657    y
Name: band, dtype: object
2 3
202207-08
visitId
159471    g
159478    g
162699    r
167864    y
169838    y
169839    y
Name: band, dtype: object
3 4
202208-09
visitId
174534    i
174535    i
174579    i
179999    u
180000    u
180001    u
181869    r
181901    r
183772    g
183818    g
189315    y
189317    y
189382    y
190281    y
190282    y
192355    i
Name: band, dtype: object
4 5
202209-10
visitId
193111    r
193144    r
193147    r
193189    r
193233    r
193235    r
193826    g
193849    r
193881    r
193888    r
200750    u
202617    r
204708    i
206039    y
206040    y
206073    y
Name: band, dtype: object
5 6
202210-11
visitId
207792    y
209015    z
209031    z
209032    z
209063    z
209080    z
209082    z
211099    i
211100    i
211198    i
211228    i
211478    i
211484    i
211490    i
211527    i
211531    i
212071    r
212085    r


In [27]:
for i in range(0, len(dates_calexps_ym)-1):
    print(i, i+1)
    print(f"{dates_calexps_ym['year'][i]+dates_calexps_ym['month'][i]}-{dates_calexps_ym['month'][i+1]}")
    my_range = np.array((df['obsStartMJD'] >= dates_calexps_ym['mjd'][i]) & 
                    (df['obsStartMJD'] < dates_calexps_ym['mjd'][i+1]))
    my_visits = df[my_range].groupby(["band"]).size() #agg({'g':'count', 'i':'count', 'r':'count', 'u':'count', 'y':'count', 'z':'count'})
    print(my_visits)

0 1
202201-02
band
y    4
dtype: int64
1 2
202202-07
band
y    2
dtype: int64
2 3
202207-08
band
g    2
r    1
y    3
dtype: int64
3 4
202208-09
band
g    2
i    4
r    2
u    3
y    5
dtype: int64
4 5
202209-10
band
g     1
i     1
r    10
u     1
y     3
dtype: int64
5 6
202210-11
band
g     4
i    12
r    10
y     1
z     6
dtype: int64
6 7
202211-12
band
g    2
i    9
r    4
z    3
dtype: int64


In [ ]:
# 'g':'count', 'i':'count', 'r':'count', 'u':'count', 'y':'count', 'z':'count'

In [20]:
# create the coadf for each month
# for i in range(len(dates_calexps_ym)-1):
for i in range(2):

    my_range = np.array((visitTable['obsStartMJD'] >= dates_calexps_ym['mjd'][i]) & 
                    (visitTable['obsStartMJD'] < dates_calexps_ym['mjd'][i+1]))
    my_visits = visitTable_uni.visit[my_range]
    my_visits_tupleString = "("+",".join(my_visits.astype(str))+")"
    print(my_visits_tupleString)

    my_collection_identifier = f"{my_collection_identifier_step1step2}_coadd_582025_1_{dates_calexps_ym['year'][i]+dates_calexps_ym['month'][i]}-{dates_calexps_ym['month'][i+1]}"
    # print(my_collection_identifier)

    my_outputCollection = f"u/{user}_pm/{my_collection_identifier}" 
    print('Name of new butler collection for my output: ', my_outputCollection)


    print(repo)
    print(collections_step1step2)
    print(my_outputCollection)
    simpleButler = SimplePipelineExecutor.prep_butler(repo, 
                                                  inputs=[collections_step1step2], 
                                                  output=my_outputCollection)
    print(simpleButler.registry.getCollectionChain(my_outputCollection))


    yaml_file = '$DRP_PIPE_DIR/pipelines/LSSTCam-imSim/DRP-test-med-1.yaml'
    steps = 'makeWarp,selectDeepCoaddVisits,assembleCoadd,detection,mergeDetections,deblend,measure'
    my_uri = yaml_file + '#' + steps
    print(my_uri)

    assembleCoaddPipeline = Pipeline.from_uri(my_uri)

    queryString = f"visit in {my_visits_tupleString} AND skymap = 'DC2'"
    print(queryString)


    spe = SimplePipelineExecutor.from_pipeline(assembleCoaddPipeline, 
                                           where=queryString, 
                                           butler=simpleButler)


    quanta = spe.run()

    del simpleButler
    # del assembleCoaddPipeline
    # del spe
    # del quanta

(12453,12454,12466,12482)
Name of new butler collection for my output:  u/taceroc_pm/test_512025_1_t4026p15y2022_coadd_582025_1_202201-02
/global/cfs/cdirs/lsst/production/gen3/DC2/Run2.2i/repo
u/taceroc_pm/test_512025_1_t4026p15y2022
u/taceroc_pm/test_512025_1_t4026p15y2022_coadd_582025_1_202201-02
('u/taceroc_pm/test_512025_1_t4026p15y2022_coadd_582025_1_202201-02/20250508T193358Z', 'u/taceroc_pm/test_512025_1_t4026p15y2022')
$DRP_PIPE_DIR/pipelines/LSSTCam-imSim/DRP-test-med-1.yaml#makeWarp,selectDeepCoaddVisits,assembleCoadd,detection,mergeDetections,deblend,measure
visit in (12453,12454,12466,12482) AND skymap = 'DC2'


/cvmfs/sw.lsst.eu/linux-x86_64/lsst_distrib/v27.0.0/conda/envs/lsst-scipipe-8.0.0-exact-ext/share/eups/Linux64/obs_lsst/g5c5439fafc+141fc35c6f/config/assembleCoadd.py:28: FutureWarning: Config field doSigmaClip is deprecated: True
  config.doSigmaClip = False


(37656,37657)
Name of new butler collection for my output:  u/taceroc_pm/test_512025_1_t4026p15y2022_coadd_582025_1_202202-07
/global/cfs/cdirs/lsst/production/gen3/DC2/Run2.2i/repo
u/taceroc_pm/test_512025_1_t4026p15y2022
u/taceroc_pm/test_512025_1_t4026p15y2022_coadd_582025_1_202202-07
('u/taceroc_pm/test_512025_1_t4026p15y2022_coadd_582025_1_202202-07/20250508T211058Z', 'u/taceroc_pm/test_512025_1_t4026p15y2022')
$DRP_PIPE_DIR/pipelines/LSSTCam-imSim/DRP-test-med-1.yaml#makeWarp,selectDeepCoaddVisits,assembleCoadd,detection,mergeDetections,deblend,measure
visit in (37656,37657) AND skymap = 'DC2'


/cvmfs/sw.lsst.eu/linux-x86_64/lsst_distrib/v27.0.0/conda/envs/lsst-scipipe-8.0.0-exact-ext/share/eups/Linux64/obs_lsst/g5c5439fafc+141fc35c6f/config/assembleCoadd.py:28: FutureWarning: Config field doSigmaClip is deprecated: True
  config.doSigmaClip = False
Traceback (most recent call last):
  File "/cvmfs/sw.lsst.eu/linux-x86_64/lsst_distrib/v27.0.0/conda/envs/lsst-scipipe-8.0.0-exact-ext/share/eups/Linux64/meas_extensions_scarlet/g1850869495+d7bfc87f51/python/lsst/meas/extensions/scarlet/scarletDeblendTask.py", line 946, in deblend
    blend, skippedSources, skippedBands = deblend(
                                          ^^^^^^^^
  File "/cvmfs/sw.lsst.eu/linux-x86_64/lsst_distrib/v27.0.0/conda/envs/lsst-scipipe-8.0.0-exact-ext/share/eups/Linux64/meas_extensions_scarlet/g1850869495+d7bfc87f51/python/lsst/meas/extensions/scarlet/scarletDeblendTask.py", line 137, in deblend
    observation = buildObservation(
                  ^^^^^^^^^^^^^^^^^
  File "/cvmfs/sw.lsst.eu/linux-x86_6